In [2]:
from scipy.optimize import fsolve
import numpy as np

$$
PV_{\text{Default Leg}} = LGD \times \sum_{i=1}^{n3Y} P(t, t_i) \left[ e^{-\lambda_{3Y} \cdot (t_{i-1} - t)} - e^{-\lambda_{3Y} \cdot (t_i - t)} \right]
$$

$$
PV_{\text{Coupon Leg}} = s \times \sum_{i=1}^{n3Y} \alpha_i \cdot P(t, t_i) \cdot e^{-\lambda_{3Y} \cdot (t_i - t)}
$$


$$
UF^{3Y} = PV_{\text{Default Leg}} - PV_{\text{Coupon Leg}}
$$

In [3]:
# Tasso di interesse zero-coupon annuale
r = 0.03

# Tasso di recupero
recovery_rate = 0.4

# Loss Given Default (1 - recovery rate)
LGD = 1 - recovery_rate

# Spread del CDS, che è pari al tasso di cedola annuale
spread = 0.01

# Numero di periodi di pagamento fino alla scadenza del CDS a 3 anni
n3Y = 3

# Premio upfront osservato per il CDS a 3 anni
upfront_3y = 0.068

upfront_5y = 0.096



In [4]:
# Funzione che calcola il prezzo di uno zero-coupon bond che matura al tempo t
def zero_coupon_bond_price(r, t):
    return np.exp(-r * t)

In [5]:
# Dati aggiuntivi per il problema
upfront_observed_3y = upfront_3y  # Premio upfront osservato per il CDS a 3 anni

# Funzione che calcola il valore attuale del premio upfront per un CDS a 3 anni secondo la formula fornita
def pv_upfront_3y_new(lambda_3Y):
    # Calcolo del valore attuale dei flussi di cassa per la perdita data il default (LGD)
    pv_loss = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_3Y * (i - 1)) - np.exp(-lambda_3Y * i)) for i in range(1, n3Y + 1)])
    
    # Calcolo del valore attuale dei flussi di cassa dei coupon pagati
    pv_coupon_payments = sum([spread * zero_coupon_bond_price(r, i) * np.exp(-lambda_3Y * i) for i in range(1, n3Y + 1)])
    
    # Valore attuale del premio upfront calcolato
    pv_upfront_calculated = LGD * pv_loss - pv_coupon_payments
    
    # La differenza tra il valore attuale calcolato e il premio upfront osservato
    return pv_upfront_calculated - upfront_observed_3y

lambda_3Y_initial_guess = 0.1
# Risolviamo per lambda_3Y che zera la funzione pv_upfront_3y_new
lambda_3Y_solution = fsolve(pv_upfront_3y_new, lambda_3Y_initial_guess)

lambda_3Y_solution[0]*100



5.993478603980348

$$
% Definizione del valore attuale del premio upfront per il CDS a 5 anni
PV_{\text{upfront, 5y}}(\lambda_{5Y}) = LGD \left( \sum_{i=1}^{3} P(t, t_i) \left[ e^{-\lambda_{3Y} \cdot (t_{i-1} - t)} - e^{-\lambda_{3Y} \cdot (t_i - t)} \right] + \sum_{i=4}^{5} P(t, t_i) \left[ e^{-\lambda_{5Y} \cdot (t_{i-1} - t)} - e^{-\lambda_{5Y} \cdot (t_i - t)} \right] \right) - s \left( \sum_{i=1}^{3} \alpha_i P(t, t_i) e^{-\lambda_{3Y} \cdot (t_i - t)} + \sum_{i=4}^{5} \alpha_i P(t, t_i) e^{-\lambda_{5Y} \cdot (t_i - t)} \right)
\\
\\
% Condizione che il valore attuale calcolato sia uguale al premio upfront osservato sul mercato
UF^{5Y} = PV_{\text{upfront, 5y}}(\lambda_{5Y})
$$

In [6]:
# Numero di periodi di pagamento fino alla scadenza del CDS a 5 anni
n5Y = 5

# Premio upfront osservato per il CDS a 5 anni
upfront_observed_5y = upfront_5y  # Già definito precedentemente come 0.096

# Calcoliamo il valore attuale dei flussi di cassa fino a 3 anni usando lambda_3Y trovato
lambda_3Y = lambda_3Y_solution[0]  # Utilizziamo il lambda_3Y trovato precedentemente

# Funzione per il calcolo del valore attuale del premio upfront per il CDS a 5 anni
def pv_upfront_5y_new(lambda_5Y):
    # Flussi di cassa fino a 3 anni, usando lambda_3Y
    pv_loss_3y = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_3Y * (i - 1)) - np.exp(-lambda_3Y * i)) for i in range(1, n3Y + 1)])
    pv_coupon_payments_3y = sum([spread * zero_coupon_bond_price(r, i) * np.exp(-lambda_3Y * i) for i in range(1, n3Y + 1)])
    
    # Aggiungiamo i flussi di cassa dagli anni 3 a 5, usando lambda_5Y
    pv_loss_5y = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_5Y * (i - 1)) - np.exp(-lambda_5Y * i)) for i in range(n3Y + 1, n5Y + 1)])
    pv_coupon_payments_5y = sum([spread * zero_coupon_bond_price(r, i) * np.exp(-lambda_5Y * i) for i in range(n3Y + 1, n5Y + 1)])
    
    # Valore attuale totale del premio upfront calcolato
    pv_upfront_calculated = (LGD * (pv_loss_3y + pv_loss_5y)) - (pv_coupon_payments_3y + pv_coupon_payments_5y)
    
    # La differenza tra il valore attuale calcolato e il premio upfront osservato per il CDS a 5 anni
    return pv_upfront_calculated - upfront_observed_5y

# Valore iniziale stimato per lambda_5Y per iniziare la ricerca della soluzione
lambda_5Y_initial_guess = 0.1

# Utilizzo di fsolve per trovare il valore di lambda_5Y
lambda_5Y_solution = fsolve(pv_upfront_5y_new, lambda_5Y_initial_guess)

lambda_5Y_solution[0] *100 


4.868322413146313

$$
% Calcolo della probabilità di sopravvivenza fino a un certo tempo T
\text{Surv}(T) = 
\begin{cases} 
e^{-\lambda_{3Y} \cdot T}, & \text{se } T \leq 3 \\
e^{-\lambda_{3Y} \cdot 3} \cdot e^{-\lambda_{5Y} \cdot (T - 3)}, & \text{se } T > 3 
\end{cases}

$$

$$
\text{Probabilità di sopravvivenza a 1 anno} = e^{-\lambda_{3Y} \cdot 1}\\
\text{Probabilità di sopravvivenza a 2 anni} = e^{-\lambda_{3Y} \cdot 2}\\
\text{Probabilità di sopravvivenza a 3 anni} = e^{-\lambda_{3Y} \cdot 3}\\
\text{Probabilità di sopravvivenza a 4 anni} = e^{-\lambda_{3Y} \cdot 3} \cdot e^{-\lambda_{5Y} \cdot (4 - 3)}\\
\text{Probabilità di sopravvivenza a 5 anni} = e^{-\lambda_{3Y} \cdot 3} \cdot e^{-\lambda_{5Y} \cdot (5 - 3)}
$$



In [17]:

# Funzione per calcolare la probabilità di sopravvivenza fino a un certo tempo T
def calculate_survival_probability(lambda_1, lambda_2, T, n3Y):
    # Se T è entro i primi 3 anni, usiamo solo lambda_1
    if T <= n3Y:
        return np.exp(-lambda_1 * T)
    # Altrimenti, usiamo lambda_1 per i primi 3 anni e lambda_2 per i restanti anni fino a T
    else:
        return np.exp(-lambda_1 * n3Y) * np.exp(-lambda_2 * (T - n3Y))

# Calcoliamo le probabilità di sopravvivenza per ciascun anno
prob_survival_1y = calculate_survival_probability(lambda_3Y, lambda_5Y_solution[0], 1, n3Y)*100
prob_survival_2y = calculate_survival_probability(lambda_3Y, lambda_5Y_solution[0], 2, n3Y)*100
prob_survival_3y = calculate_survival_probability(lambda_3Y, lambda_5Y_solution[0], 3, n3Y)*100
prob_survival_4y = calculate_survival_probability(lambda_3Y, lambda_5Y_solution[0], 4, n3Y)*100
prob_survival_5y = calculate_survival_probability(lambda_3Y, lambda_5Y_solution[0], 5, n3Y)*100

(prob_survival_1y, prob_survival_2y, prob_survival_3y, prob_survival_4y, prob_survival_5y)


(94.18259517816966,
 88.70361234494986,
 83.54336412325705,
 79.57361783953675,
 75.7925027621665)

---

In [10]:
# r = 0.03  # Tasso di interesse zero-coupon annuale
# recovery_rate = 0.4  # Tasso di recupero
# LGD = 1 - recovery_rate  # Loss Given Default
# spread = 0.01  # Spread del CDS, che è pari al tasso di cedola annuale
# n3Y = 3  # Numero di periodi di pagamento fino alla scadenza del CDS a 3 anni
# n5Y = 5  # Numero di periodi di pagamento fino alla scadenza del CDS a 5 anni
# upfront_observed_3y = 0.068  # Premio upfront osservato per il CDS a 3 anni
# upfront_observed_5y = 0.096  # Premio upfront osservato per il CDS a 5 anni

# # Definisco la funzione per il prezzo di uno zero-coupon bond
# def zero_coupon_bond_price(r, t):
#     return np.exp(-r * t)

# Definisco la funzione per calcolare il valore attuale del premio upfront per il CDS a 3 anni
def pv_upfront_3y(lambda_3Y):
    pv_loss = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_3Y * (i - 1)) - np.exp(-lambda_3Y * i)) for i in range(1, n3Y + 1)])
    pv_cedole = sum([spread * zero_coupon_bond_price(r, i) * np.exp(-lambda_3Y * i) for i in range(1, n3Y + 1)])
    return LGD * pv_loss - pv_cedole - upfront_observed_3y

# Calcolo lambda_3Y usando fsolve
lambda_3Y_initial_guess = 0.1
lambda_3Y_solution = fsolve(pv_upfront_3y, lambda_3Y_initial_guess)
lambda_3Y = lambda_3Y_solution[0]

# Definisco la funzione per calcolare il valore attuale del premio upfront per il CDS a 5 anni
def pv_upfront_5y(lambda_5Y):
    pv_loss_3y = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_3Y * (i - 1)) - np.exp(-lambda_3Y * i)) for i in range(1, n3Y + 1)])
    pv_cedole_3y = sum([spread * zero_coupon_bond_price(r, i) * np.exp(-lambda_3Y * i) for i in range(1, n3Y + 1)])
    pv_loss_5y = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_5Y * (i - 1)) - np.exp(-lambda_5Y * i)) for i in range(n3Y + 1, n5Y + 1)])
    pv_cedole_5y = sum([spread * zero_coupon_bond_price(r, i) * np.exp(-lambda_5Y * i) for i in range(n3Y + 1, n5Y + 1)])
    return LGD * (pv_loss_3y + pv_loss_5y) - (pv_cedole_3y + pv_cedole_5y) - upfront_observed_5y

# Calcolo lambda_5Y usando fsolve
lambda_5Y_initial_guess = 0.1
lambda_5Y_solution = fsolve(pv_upfront_5y, lambda_5Y_initial_guess)
lambda_5Y = lambda_5Y_solution[0]


# lambda_3Y, lambda_5Y 

# Calcolo e stampo i valori del premio upfront calcolati
upfront_3y_calculated = pv_upfront_3y(lambda_3Y) + upfront_observed_3y
upfront_5y_calculated = pv_upfront_5y(lambda_5Y) + upfront_observed_5y

upfront_3y_calculated, upfront_5y_calculated


(0.06800000000000013, 0.09599999999999999)

In [15]:
lambda_3Y*100, lambda_5Y*100

(5.993478603980348, 4.868322413146313)

In [12]:
upfront_observed_3y, upfront_3y_calculated, upfront_3y_calculated-upfront_observed_3y

(0.068, 0.06800000000000013, 1.249000902703301e-16)

In [13]:
upfront_observed_5y, upfront_5y_calculated, upfront_5y_calculated-upfront_observed_5y

(0.096, 0.09599999999999999, -1.3877787807814457e-17)